In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os
import sys

project_pth = os.path.join(os.getcwd(), '..', '..')  
sys.path.append(project_pth)

from utils.transformation import reusable





### **DimUser**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


### **Autoloader**

In [0]:
df_user = (spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "parquet")
    .option("cloudFiles.schemaLocation", "abfss://silver@spotifyadls1001.dfs.core.windows.net/DimUser/checkpoint")
    .option("schemaEvolutionMode", "addNewColumns") 
    .load("abfss://bronze@spotifyadls1001.dfs.core.windows.net/DimUser")
)



In [0]:
df_user = df_user.withColumn("user_name", upper(col("user_name")))
display(df_user)

In [0]:
df_user_obj = reusable()

df_user = df_user_obj.dropColumns(df_user, ['_rescued_data'])
df_user = df_user.dropDuplicates(['user_id'])
display(df_user)

In [0]:
(
    df_user.writeStream
        .format("delta")
        .outputMode("append")
        .option("checkpointLocation", "abfss://silver@spotifyadls1001.dfs.core.windows.net/DimUser/checkpoint")
        .trigger(once = True)
        .option("path", "abfss://silver@spotifyadls1001.dfs.core.windows.net/DimUser/data")    
        .toTable("spotify_cata.silver.DimUser")
)


### **DimArtist**

In [0]:
df_artist = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "parquet")
    .option("cloudFiles.schemaLocation", "abfss://silver@spotifyadls1001.dfs.core.windows.net/DimArtist/checkpoint")
    .option("schemaEvolutionMode", "addNewColumns")
    .load("abfss://bronze@spotifyadls1001.dfs.core.windows.net/DimArtist")
)
display(df_artist)

In [0]:
df_art_obj = reusable()

df_artist = df_art_obj.dropColumns(df_artist, ['_rescued_data'])
df_artist = df_artist.dropDuplicates(['artist_id'])
display(df_artist)

In [0]:
( df_artist.writeStream.format("delta")
            .outputMode("append")
            .option("checkpointLocation", "abfss://silver@spotifyadls1001.dfs.core.windows.net/DimArtist/checkpoint") 
            .trigger(once = True)
            .option("path", "abfss://silver@spotifyadls1001.dfs.core.windows.net/DimArtist/data")    
            .toTable("spotify_cata.silver.DimArtist") 
            )         

In [0]:
df_track = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "parquet")
    .option("cloudFiles.schemaLocation", "abfss://silver@spotifyadls1001.dfs.core.windows.net/DimTrack/checkpoint")
    .load("abfss://bronze@spotifyadls1001.dfs.core.windows.net/DimTrack")
)
display(df_track)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:

 df_track = df_track.withColumn("durationFlag", when(col("duration_sec")<150,"low")\
                                                .when(col("duration_sec")<300, "medium")\
                                                .otherwise("high"))
 df_track = df_track.withColumn("track_name", regexp_replace(col("track_name"), "-", " "))
 df_track = reusable().dropColumns(df_track, ['_rescued_data'])

display(df_track)                                                    

In [0]:
df_track.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "abfss://silver@spotifyadls1001.dfs.core.windows.net/DimTrack/checkpoint") \
    .trigger(once=True) \
    .option("path", "abfss://silver@spotifyadls1001.dfs.core.windows.net/DimTrack/data") \
    .toTable("spotify_cata.silver.DimTrack")


### **DimDate**

In [0]:
df_date = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "parquet")
    .option("cloudFiles.schemaLocation", "abfss://silver@spotifyadls1001.dfs.core.windows.net/DimDate/checkpoint")
    .load("abfss://bronze@spotifyadls1001.dfs.core.windows.net/DimDate")
)
display(df_date)

In [0]:
df_date = reusable().dropColumns(df_date, ['_rescued_data'])

(df_date.writeStream 
    .format("delta") 
    .outputMode("append")
    .option("checkpointLocation", "abfss://silver@spotifyadls1001.dfs.core.windows.net/DimDate/checkpoint") 
    .trigger(once=True) 
    .option("path", "abfss://silver@spotifyadls1001.dfs.core.windows.net/DimDate/data") 
    .toTable("spotify_cata.silver.DimDate")
)


## **FactStream**

In [0]:
df_fact = (spark.readStream.format("cloudFiles")\
                .option("cloudFiles.format", "parquet")\
                .option("cloudFiles.schemaLocation", "abfss://silver@spotifyadls1001.dfs.core.windows.net/FactStream/checkpoint")
                .option("schemaEvolutionMode", "addNewColumns")
                .load("abfss://bronze@spotifyadls1001.dfs.core.windows.net/FactStream")
) 
display(df_fact)   

In [0]:
df_fact = reusable().dropColumns(df_fact,["_rescued_data"])


In [0]:
..(df_fact.writeStream.format("delta")
    .outputMode("append")
    .option("checkpointLocation", "abfss://silver@spotifyadls1001.dfs.core.windows.net/FactStream/checkpoint")
    .trigger(once=True)
    .option("path", "abfss://silver@spotifyadls1001.dfs.core.windows.net/FactStream/data")
    .toTable("spotify_cata.silver.FactStream")
 
 
 )
